# Understanding TARDIS Plasma Physics: From Atoms to Spectra

> **Note:** We strongly recommend turning off code autocompletion for this notebook. This exercise is designed to engage your understanding of TARDIS plasma physics and atomic data, rather than relying on automated suggestions. In VSCode, you can temporarily disable autocomplete in notebook cells by navigating to **Settings → Notebook: Suggest Enabled** and unchecking the box, or by creating a workspace with a `.vscode/settings.json` file containing `"notebook.suggest.enabled": false`. This will help you focus on understanding the physics behind each step.

In this notebook, we'll explore how TARDIS calculates the physical conditions inside supernova ejecta. You'll learn about atomic data, ionization states, excitation levels, and how these connect to the spectral lines we observe in supernovae.

In [ ]:
from matplotlib import pyplot as plt
from tardis.io.configuration.config_reader import Configuration
from tardis import run_tardis
from astropy import units as u

In [ ]:
from tardis.io.atom_data import download_atom_data, AtomData
# Download the Kurucz CD23 atomic data This is a basic atomic data set. Will not download if already present.
download_atom_data('kurucz_cd23_chianti_H_He_latest')

## Starting with a Converged Simulation

We saw how to ensure TARDIS converges properly in the previous notebook. Here, we'll start with a well-converged simulation so we can focus on understanding the plasma physics.

**$\blacktriangleright$ TASK**  Load in the config and run the simulation again by executing the two cells below. Recall what each of the values we're changing updates in the simulation. 

<div style="background-color: #e3f2fd; border-left: 4px solid #2196f3; padding: 10px; margin: 10px 0;">
<strong>**Note:**</strong> The configuration below uses the convergence parameters we learned about in the previous notebook to ensure reliable results.
</div>

In [ ]:
conf = Configuration.from_yaml('tardis_example.yml') 
conf.montecarlo.no_of_packets=1e4 
conf.montecarlo.iterations = 10  
conf.montecarlo.last_no_of_packets = 1e5
conf.montecarlo.convergence_strategy.stop_if_converged = True
conf.montecarlo.convergence_strategy.hold_iterations = 4 


In [ ]:
sim = run_tardis(conf, show_convergence_plots=True)

## Atomic Data: The Foundation of Plasma Physics

One TARDIS input that we've seen but haven't focused on is the **Atomic Data**. This is generated by the sister package [CARSUS](https://tardis-sn.github.io/carsus/), which creates novel combinations of atomic data from various sources. 

**$\blacktriangleright$ TASK** - Before we dive into the plasma physics, let's understand what atomic data TARDIS needs:

1. **Think about it:** What atomic properties would you need to calculate ionization states?
2. **Consider:** What information is required to determine which spectral lines can form?

The atomic data provides all the information the simulation needs about the elements in the ejecta: elemental masses, ionization energies, excitation energies, and line transition probabilities.

## Double click to edit this cell, and put your thoughts down here

Add solution cell - we need ionization energies, excitation energies, line info (oscillator strengths)

In [ ]:
#NOTE - we do not need to load this as a standalone object for a TARDIS simulation, 
# but we can do so to inspect the data before it's used if we want.
atomic_data = AtomData.from_hdf('kurucz_cd23_chianti_H_He_latest.h5')

The atomic data provides all of the information that the simulation needs about the elements in the ejecta. This includes elemental masses to help us convert mass fractions into actual number densities, but perhaps the most important parts are the information about the energies associated with ionizing and exciting the atoms, and information about the line transitions between excited states. Let's take a look. 

In [ ]:
atomic_data.levels

In [ ]:
atomic_data.ionization_data

### Exploring Ionization Energies

**$\blacktriangleright$ TASK** - Let's verify some basic atomic physics using the loaded atomic data.

The energy required to remove one electron from neutral Oxygen (O I → O II) is about 13.6 eV. See if you can verify this using the atomic data we've loaded.

**Your task:** Find the ionization energy for Oxygen and convert it to eV to check against the expected value.

In [ ]:
# Start by grabbing the appropriate ionization energy, and then convert it to the appropriate units. 
# Remember that TARDIS always stores units internally in the CGS unit system.
# Hint: Look for Oxygen (atomic number 8) going from ion_number 0 to 1

# Your code here to find and convert the ionization energy
oxygen_ionization_energy = # Fill this in

# Convert to eV using astropy units
ionization_energy_eV = # Fill this in

ionization_energy_eV

Now let's take a look at the lines information which might be new to you. This dataframe contains all the information we need to talk about how probable an individual line transition is. Specifically, this now includes an "oscillator strength" which appears as f_ul (describing the transition from the less excited state to the more excited state, or lower to upper) and f_lu (upper to lower instead). Remember that, ultimately, we care about opacities. Once we have the number densities of atoms in their specific excitation and ionization state, we need a cross section to calculate the line opacity of the plasma. 

The cross section of a absorption line is given by 

$\sigma_{line} = \frac{\pi e^2}{m_e c}f_{lu}$

You won't need to evaluate this here, but just know that the lines information contains the $f_{lu}$ values we need to go from the number densities we need to the opacities we care about.

In [ ]:
atomic_data.lines

## Understanding the simulation

Now let's take a look at what's contained in a TARDIS simulation, once you have something you want to look at more deeply. 

There are a couple different objects that contain different parts of the code. The first, most general one, is what we call the "simulation state." This object contains a lot of the physical properties of the supernova ejecta that are stored in the "composition" and "geometry" objects.

Let's take a look.

In [ ]:
geometry = sim.simulation_state.geometry
composition = sim.simulation_state.composition

### Understanding the Simulation Structure

The geometry object contains all the information about the physical size and location of the different ejecta shells. This is crucial because TARDIS calculates physical properties independently for each shell.

**$\blacktriangleright$ Key Insight** - TARDIS uses a shell-based approach:

- Each shell has constant density and temperature
- Most physical properties are computed per shell
- For our example configuration: **20 shells** = 20 data points for most quantities


In [ ]:
geometry.no_of_shells

In [ ]:
len(geometry.volume), len(geometry.r_inner), len(geometry.v_outer)

TARDIS works formally on a velocity grid. This is largely a result of being a code built assuming homologous expansion, which we've talked about on Monday. To briefly recap, we assume that no net forces are acting on the ejecta, so the ejecta is not accelerating and the structure is frozen in. This lets us, among other things, easily evolve the ejecta profile to any given time since the velocity of each shell will not change. In any case, it is often useful to look at whatever parameter you're interested in plotted against v_inner (or another shell dependent velocity like v_outer) to see how the parameter changes over the ejecta.

**$\blacktriangleright$ TASK** - Explore the relationship between shell structure and physical scales:

1. **Examine the geometry** - How do shell volumes change with velocity?
2. **Consider the physics** - Why might outer shells have different properties?
3. **Think about observations** - How does this relate to what we see in spectra?

After you've thought about this for a minute, the next two cells will show you how you can look at these properties in the TARDIS simulation. Do they match your expectations? Why or why not?

In [ ]:
plt.plot(geometry.v_inner, geometry.volume, drawstyle='steps')
plt.xlabel('Inner Velocity [cm / s]')
plt.ylabel('Shell Volume [cm$^3$]')

In [ ]:
plt.plot(geometry.v_inner, composition.density, drawstyle='steps')
plt.xlabel("Inner Velocity [cm / s]")
plt.ylabel('Shell Density [g cm$^{-3}$]')

We can also look at how the different elements are broken down in each shell. We asked this simulation to have uniform elemental mass fractions across the different shells, but some simulations might be more complex like we'll see tomorrow.

In [ ]:
composition.elemental_mass_fraction

In [ ]:
sim.convergence_plots.plasma_plot

### **TASK**: Recreate the Final Iteration of the Temperature Plot

Now make sure you understand how to access the simulation data! 

**$\blacktriangleright$ Your Challenge** - Recreate the final iteration of the radiative temperature plot from the convergence plots. The simulation object retains the information from the final iteration, and you should be able to plot the same radiative temperature versus velocity structure.

**Hints:**
- Use `plt.plot()` with `drawstyle='steps'` to match the convergence plot style
- Look for temperature and velocity data in the simulation object
- The final converged values should match what you saw in the convergence plot

In [ ]:
# Construct the radiative temperature plot for the final iteration
# The simulation object has the final converged values saved

# Your code here - plot radiative temperature vs velocity
# Use drawstyle='steps' to match the convergence plot style
plt.plot(?, ?, drawstyle='steps')
plt.xlabel('Velocity [cm/s]')
plt.ylabel('Radiative Temperature [K]')
plt.title('Final Radiative Temperature Profile')

In [ ]:
# Solution
# Plot the final radiative temperature vs velocity
plt.plot(geometry.v_inner, sim.plasma.t_rad, drawstyle='steps')
plt.xlabel('Velocity [cm/s]')
plt.ylabel('Radiative Temperature [K]')
plt.title('Final Radiative Temperature Profile')
plt.grid(True, alpha=0.3)

# Inspecting the Plasma

Now let's dive in to one of the two most important parts of the simulation. The plasma object exists to calculate the number densities that we've talked so much about and is very similar to what we saw yesterday. Let's start by grabbing saving it to a new variable so we can play with it easily.

In [ ]:
plasma = sim.plasma

All the ion number densities are held in the dataframe in the next cell. Notice that now the columns correspond to each of the shells.

In [ ]:
plasma.ion_number_density

Let's do a quick pandas operation to normalize that number densities, just so that we can more easily understand what's going on in each shell. 

In [ ]:
normalized_ions = plasma.ion_number_density / plasma.ion_number_density.groupby('atomic_number').sum()

In [ ]:
# This cell will plot the normalized ionization densities of Oxygen, Magnesium, and Argon. 
SHELL_ID = 15 #Pick a shell to take a look at. The current model has shells labeled 0-19

plt.plot(normalized_ions.loc[8][SHELL_ID],  alpha=1, label='Oxygen')
plt.plot(normalized_ions.loc[12][SHELL_ID], alpha=1,  label='Magnesium')
plt.plot(normalized_ions.loc[18][SHELL_ID], alpha=1,  label='Argon')
plt.legend()
plt.xlim(0, 5)
plt.xlabel('Ionization State')
plt.ylabel('Fractional Population')

### Ionization Patterns Across the Ejecta

Notice that in shell 15, the different elements are almost entirely in one ionization state or another. This is very important for understanding why certain lines may or may not appear in a supernova spectrum, but as we'll see later are not necessarily the whole story. 

**$\blacktriangleright$ Physical Insight** - Why do we see such sharp ionization transitions?

- **Temperature effects:** Higher temperature → more ionization
- **Density effects:** Higher density → more recombination
- **Radiation field:** Photoionization vs. thermal ionization

Let's examine how the ionization front changes across the ejecta by looking at Argon's behavior.

# 

In [ ]:
ATOMIC_NUMBER = 18 #Picking Argon

for column in normalized_ions.columns:
    plt.plot(normalized_ions.loc[ATOMIC_NUMBER][column], label=column)
    
plt.xlim(0,4)
plt.legend()
plt.xlabel('Ionization State')
plt.ylabel('Fractional Population')
plt.title('Argon Ionization State')

### **TASK**: Analyze Oxygen Ionization

**$\blacktriangleright$ Compare and Contrast** - Now let's examine how Oxygen behaves compared to Argon.

**Your task:** Create a similar plot for Oxygen and analyze the differences:

1. **Make the plot** using the code structure from the Argon example above
2. **Observe the pattern** - How does Oxygen's ionization change across shells?
3. **Physical reasoning** - Why might Oxygen behave differently than Argon?

In [ ]:
# Your code here - create an Oxygen ionization plot similar to the Argon one above
ATOMIC_NUMBER = 8  # Oxygen

# Plot the ionization states across different shells
for column in normalized_ions.columns:
    plt.plot(normalized_ions.loc[ATOMIC_NUMBER][column], label=column)
    
plt.xlim(0,4)
plt.legend()
plt.xlabel('Ionization State')
plt.ylabel('Fractional Population')
plt.title('Oxygen Ionization State Across Shells')

Let's also look at the excitation level densities, and normalize them again so we can put them on a reasonable scale and not worry about the overall density decreasing as we move out through the ejecta.

In [ ]:
normalized_levels = plasma.level_number_density / plasma.level_number_density.groupby(['atomic_number', 'ion_number']).sum()

In [ ]:
SHELL_ID = 10 #Picking the 10th SHELL for plotting. You can change this if you want to see another shell.
ATOMIC_NUMBER = 14 # Picking Si

plt.plot(normalized_levels.loc[ATOMIC_NUMBER,0][SHELL_ID], label='Si I')
plt.plot(normalized_levels.loc[ATOMIC_NUMBER,1][SHELL_ID], label='Si II')
plt.plot(normalized_levels.loc[ATOMIC_NUMBER,2][SHELL_ID], label='Si III')

plt.legend()
plt.xlim(0, 6)
plt.ylabel('Excitation Fractional Number Density')
plt.xlabel('Excitation Level')

But ultimately what matters is the actual number densities, so let's go back to looking at them directly. 

We can focus again on the $\lambda\lambda$ Si II 6355 $\AA$ doublet. We saw earlier that the $\lambda\lambda$ Si 6355 $\AA$ doublet is two blended lines at 6348 and 6373 $\AA$, transitioning from excited level 7 to 15 and 7 to 13. 

In [ ]:
plasma.level_number_density.loc[ATOMIC_NUMBER, 0][SHELL_ID]

In [ ]:
# IN HERE DIVIDE BY G???

SHELL_ID = 10 #Picking the 10th shell for plotting
ATOMIC_NUMBER = 14 # Picking Si

plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 0][SHELL_ID], label='Silicon I')
plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 1][SHELL_ID], label='Silicon II')
plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 2][SHELL_ID], label='Silicon III')
plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 3][SHELL_ID], label='Silicon IV')

plt.legend()
plt.yscale('log')
plt.xlabel('Excitation Level')
plt.ylabel('Absolute Number Density')

There's a lot of interesting structure there, but we can see that for each of these states the majority of populated levels are in the first couple ones, so let's zoom into that.

In [ ]:
SHELL_ID = 10 #Picking the 10th shell for plotting
ATOMIC_NUMBER = 14 # Picking Si
#Plotting the actual level number densities for different levels of Si
plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 0][SHELL_ID], label='Silicon I')
plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 1][SHELL_ID], label='Silicon II')
plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 2][SHELL_ID], label='Silicon III')
plt.plot(plasma.level_number_density.loc[ATOMIC_NUMBER, 3][SHELL_ID], label='Silicon IV')

plt.legend()
plt.yscale('log')
plt.xlabel('Excitation Level')
plt.ylabel('Absolute Number Density')
plt.xlim(0, 12)

### Understanding Si II Line Formation

Remember, this is ultimately what matters for seeing which lines appear in the spectrum. We need to know how many particles are present to interact with photons on their way out of the ejecta.

**$\blacktriangleright$ The Si II Puzzle** - If most Silicon is in Si III, at low excitation states, how do we see strong Si II lines?

Looking at our plot, Si III dominates the number densities, yet we know Si II 6355 Å is a prominent supernova feature. The answer lies in understanding:

1. **Population vs. transition strength** - Even small populations can create strong lines
2. **Oscillator strength** - Some transitions are much more probable than others
3. **Optical depth** - The combination of number density and cross-section determines line strength

Let's investigate this by examining the oscillator strengths and optical depths.

# Towards Opacities, in TARDIS

We've seen a couple times that the opacity of a line is proportional to the number density of the line as well as the cross section, and the cross section is proportional to the oscillator strength. Let's take a look at the oscillator strengths of Si I.

In [ ]:
atomic_data.lines.loc[14,1] #This is Silicon II 

Now take a look at the lines that appear in the 7th excited state.

In [ ]:
atomic_data.lines.loc[14,1,7]

We immediately see that that oscillator strengths of the transitions to 15 and 13 are orders of magnitude higher than any of the other transitions. This is ultimately why these lines matter so much in so many different SNe Ia. We can verify this by taking a look at the optical depths calculated by the TARDIS plasma.

In [ ]:
sim.plasma.tau_sobolevs

The vast majority of lines in our dataframe have optical depths that are below 1e-10 in every single shell. What about our two Si II transitions?

In [ ]:
# Your task: Check the optical depth for the Si II transition from level 7 to 13
# Hint: Use the format sim.plasma.tau_sobolevs.loc[atomic_number, ion_number, level_lower, level_upper]

tau_si_7_to_13 = sim.plasma.tau_sobolevs.loc[14, 1, 7, 13]
tau_si_7_to_13

In [ ]:
sim.plasma.level_number_density.loc[14,1].sum(axis=0)

In [ ]:
# Now check the optical depth for the Si II transition from level 7 to 15
tau_si_7_to_15 = sim.plasma.tau_sobolevs.loc[14, 1, 7, 15]

tau_si_7_to_15

Much higher! 

Let's round all of this out by quickly plotting every optical depth for a single shell of your choice. Ultimately, the takeaway here is that certain lines are very physically favored, but you still need a reasonably high number density of the transition you're interested in to see any specific transition. 

In [ ]:
taus_with_wavelengths = sim.plasma.tau_sobolevs.join(atomic_data.lines.wavelength) 
#Just attaching the wavelength information to the taus, for easier plotting

In [ ]:
#Feel free to play with this plot too. This is just to demistify what the packets actually interact with inside the simulation, and how many lines are tracked and contribute to a spectrum in TARDIS. 
SHELL = 14 #Change this if you want.
plt.scatter(taus_with_wavelengths.wavelength, taus_with_wavelengths[SHELL], s=1)
plt.yscale('log')
plt.ylim(1e-15,1e1)
plt.xlim(3000, 9000)
plt.xlabel(r'Wavelength [$\AA$]')
plt.ylabel(rf'$\tau$ in shell {SHELL}')
plt.scatter(taus_with_wavelengths.loc[14,1,7,15].wavelength, taus_with_wavelengths.loc[14,1,7,15][SHELL], color='red', label = 'Si II 6355 transitions') #The first transition in the 6355 line
plt.scatter(taus_with_wavelengths.loc[14,1,7,13].wavelength, taus_with_wavelengths.loc[14,1,7,13][SHELL], color='red') #The second transition in the 6355 line
plt.legend()

We see our Si II doublet right there near the top of all the optical depths, and clearly the biggest ones between 6000 and 7000 $\AA$.

However, also take a second to appreciate how many lines we see in the plot. Thousands of lines with relatively low optical depths ultimately shape what we think of as the SN spectrum continuum. 

# Excellent Work! Key Concepts Mastered

Congratulations! You've successfully explored the core physics that drives TARDIS simulations. You've learned how to:

**Critical takeaways:**
- **Atomic data** provides the fundamental physics for ionization and excitation
- **Ionization states** vary dramatically across the ejecta due to changing physical conditions
- **Excitation levels** determine which transitions can produce observable spectral lines
- **Optical depths** reveal which lines will actually appear in the final spectrum
- **Number densities** and **oscillator strengths** work together to create line opacities

### **Understanding the Connection**

The Si II 6355 Å line we explored is a perfect example of how all these pieces fit together:
1. **Right ionization state** (Si II) is abundant in key regions
2. **Right excitation level** (level 7) has sufficient population
3. **Strong transitions** (high oscillator strengths) to levels 13 and 15
4. **High optical depths** make these lines prominent in the spectrum

### **Next Steps**

In the afternoon session, we'll explore:
- How Monte Carlo packets interact with this plasma
- What the final spectrum actually represents
- Different physics treatments and approximations in TARDIS

Take a well-deserved break! When you're ready, you'll dive into the Monte Carlo radiative transfer process that turns this plasma physics into observable spectra.